# Data Cleaning for Lecturer-Course Matching
This notebook cleans and prepares data for a logistic regression model to match lecturers with courses.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Read the CSV file
df = pd.read_csv('data.csv')
print(f"Original shape: {df.shape}")
df.head()

Original shape: (70, 40)


,SN,Institute name,Email address,District,"Google location (Lat, Log)",Rating,Subscribed,Status,Program Id,Program Name,...,Field of work,Time Preference.1,Expected hourly payment,Preferred areas to conduct lectures,Delivery Languages,Qualification Name,Awarding Body,Discipline,"Level (Doctorate, Masters, PostGraduate, Bachelors, HND, HNC).1",Duration to complete in days
0,1,Edith Cowan University - Sri Lanka,info@ecu.edu.lk,Colombo,https://maps.app.goo.gl/wHAoPaxxfE2FGvzJ7,4.8,True,Active,W23,Bachelor of Commerce,...,Software Engineering,Weekend,3000,"Computer Science, Business Management",English,BSc in Management and Technical Sciences,Kotelawala Defence University,Business Management,Bachelors,1460
1,2,Edith Cowan University - Sri Lanka,info@ecu.edu.lk,Colombo,https://maps.app.goo.gl/wHAoPaxxfE2FGvzJ7,4.8,True,Active,W23,Bachelor of Commerce,...,Software Engineering,Weekend,3000,"Computer Science, Business Management",English,MBA in Human Resources Management,University of Colombo,Business Management,Masters,730
2,3,Edith Cowan University - Sri Lanka,info@ecu.edu.lk,Colombo,https://maps.app.goo.gl/wHAoPaxxfE2FGvzJ7,4.8,True,Active,W23,Bachelor of Commerce,...,Software Engineering,Weekend,3000,"Computer Science, Business Management",English,BSc in Management and Technical Sciences,Kotelawala Defence University,Business Management,Bachelors,1460
3,4,Edith Cowan University - Sri Lanka,info@ecu.edu.lk,Colombo,https://maps.app.goo.gl/wHAoPaxxfE2FGvzJ7,4.8,True,Active,W23,Bachelor of Commerce,...,Software Engineering,Weekend,3000,"Computer Science, Business Management",English,MBA in Human Resources Management,University of Colombo,Business Management,Masters,730
4,5,Overseas Campus of Ceylon,info@overseascampus.edu.lk,Colombo,https://maps.app.goo.gl/rHcubsH2NJGKcPyc7,4.4,True,Active,NaN,Master of Business Administration in Supply Ch...,...,Software Engineering,Weekend,3000,"Computer Science, Business Management",English,BSc in Management and Technical Sciences,Kotelawala Defence University,Business Management,Bachelors,1460


In [3]:
df.columns

Index(['SN', 'Institute name', 'Email address', 'District',
       'Google location (Lat, Log)', 'Rating', 'Subscribed', 'Status',
       'Program Id', 'Program Name', 'Hourly Payment for Lecturer',
       'Level (Doctorate, Masters, PostGraduate, Bachelors, HND, HNC)',
       'Time Preference', 'Duration in days', 'Student Count',
       'Language Delivered', 'Subject Id', 'Subject index', 'Subject Name',
       'Number of Credits', 'Lecturer Index', 'Lecturer name',
       'Email address.1', 'District.1', 'Google location (Lat, Lon)',
       'Contact Number', 'Rating.1', 'Subscribed.1', 'Status.1',
       'Lecturing Experience in years', 'Field of work', 'Time Preference.1',
       'Expected hourly payment', 'Preferred areas to conduct lectures',
       'Delivery Languages', 'Qualification Name', 'Awarding Body',
       'Discipline',
       'Level (Doctorate, Masters, PostGraduate, Bachelors, HND, HNC).1',
       'Duration to complete in days'],
      dtype='object')

## Data Cleaning Steps

In [4]:
# 1. Remove duplicate rows
df = df.drop_duplicates()
print(f"Shape after removing duplicates: {df.shape}")

Shape after removing duplicates: (70, 40)


In [5]:
# 2. Handle missing values
print("Missing values before cleaning:")
print(df.isnull().sum())

# Fill missing values
df['Program Id'].fillna('Not Specified', inplace=True)
df['Contact Number'].fillna('Not Available', inplace=True)

print("\nMissing values after cleaning:")
print(df.isnull().sum())

Missing values before cleaning:
SN                                                                  0
Institute name                                                      0
Email address                                                       0
District                                                            0
Google location (Lat, Log)                                          0
Rating                                                              0
Subscribed                                                          0
Status                                                              0
Program Id                                                          7
Program Name                                                        0
Hourly Payment for Lecturer                                         0
Level (Doctorate, Masters, PostGraduate, Bachelors, HND, HNC)       0
Time Preference                                                     0
Duration in days                                          

/var/folders/v6/dj9r3dn51mbbgl1x9whmvfpm0000gn/T/ipykernel_7287/4135488593.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Program Id'].fillna('Not Specified', inplace=True)
/var/folders/v6/dj9r3dn51mbbgl1x9whmvfpm0000gn/T/ipykernel_7287/4135488593.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

In [7]:
# Define coordinates for institutes and lecturers
institute_coordinates = {
    'https://maps.app.goo.gl/wHAoPaxxfE2FGvzJ7': (6.9063231, 79.9021367),  # Edith Cowan University
    'https://maps.app.goo.gl/rHcubsH2NJGKcPyc7': (6.8619417, 79.9039102),  # Overseas Campus
    'https://maps.app.goo.gl/LRc8AV3QSZdKZ12W6': (6.8490683, 79.9212884),  # Summerset Campus
    'https://maps.app.goo.gl/A5nXtjFYoLTHLaCP8': (6.8822582, 79.8587858),  # British Institute
    'https://maps.app.goo.gl/gPY1GbV3vnXExAvx5': (6.9776515, 79.9292692),  # LPEC Campus
    'https://maps.app.goo.gl/B1DKQobDFjLgeMK19': (6.4670639, 79.9864941)   # European City Campus
}

lecturer_coordinates = {
    'https://maps.app.goo.gl/Cd9QmL5WtC78c7Fy8': (6.865639, 79.9029757),  # Chanaka Bandara
    'https://maps.app.goo.gl/1mKKCM2iaBWb75EB6': (7.0916913, 79.8210182),  # Akila Jayarathna
    'https://maps.app.goo.gl/cpvEfUc2q6jBtrgn9': (6.9043206, 79.9654323),  # Vimukthi Sripa
    'https://maps.app.goo.gl/E6Cq1GZTzAavxonn9': (6.9565152, 79.9204436),  # Gangadara Athukorala
    'https://maps.app.goo.gl/fnRJMzcMEEZ4JCHS7': (6.7671721, 79.8479364),  # Yehasha Harshani
    'https://maps.app.goo.gl/6q7frx52FuP2g1yN9': (6.9831861, 81.0793705),  # Chirantha Kithulwatta
    'https://maps.app.goo.gl/oEZxvE9Fm7huf1rS8': (6.9612784, 80.2101369),  # Maneesha Dedigama
    'https://maps.app.goo.gl/ibNUv1TAwZFbtMAS8': (6.8213291, 80.0415729),  # Akila Adithya
    'https://maps.app.goo.gl/ALj4b2p32c8fzag56': (6.8986078, 79.8686697),  # Sherina Sally
    'https://maps.app.goo.gl/AJfi6ivyv49QBksW6': (7.1114818, 79.8964773),  # Sajani Bimsara
    'https://maps.app.goo.gl/7oMp9mvN65aLe1pP9': (6.7059054, 80.0688587)   # Prof. Prathiba
}

In [8]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate distance between two points using Haversine formula
    Returns distance in kilometers
    """
    from math import radians, sin, cos, sqrt, atan2
    
    R = 6371  # Earth's radius in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    
    return distance

In [9]:
# Calculate distances
distances = []
for idx, row in df.iterrows():
    try:
        inst_url = row['Google location (Lat, Log)']
        lect_url = row['Google location (Lat, Lon)']
        
        if inst_url in institute_coordinates and lect_url in lecturer_coordinates:
            inst_lat, inst_lon = institute_coordinates[inst_url]
            lect_lat, lect_lon = lecturer_coordinates[lect_url]
            distance = calculate_distance(inst_lat, inst_lon, lect_lat, lect_lon)
        else:
            distance = None
    except:
        distance = None
    distances.append(distance)

df['distance_to_institute'] = distances

In [10]:
# 3. Extract relevant features for the model
relevant_features = [
    'Program Name',
    'Hourly Payment for Lecturer',
    'Level (Doctorate, Masters, PostGraduate, Bachelors, HND, HNC)',
    'Time Preference',
    'Student Count',
    'Subject Name',
    'Number of Credits',
    'Rating',  # Institute rating
    'Lecturer Index',
]

df_cleaned = df[relevant_features].copy()

In [11]:
# Rename columns to shorter names in snake case
column_mapping = {
    'Program Name': 'program',
    'Hourly Payment for Lecturer': 'hourly_pay',
    'Level (Doctorate, Masters, PostGraduate, Bachelors, HND, HNC)': 'level',
    'Time Preference': 'time_pref',
    'Student Count': 'student_count',
    'Subject Name': 'subject',
    'Number of Credits': 'credits',
    'Rating': 'institute_rating',
    'Lecturer Index': 'lecturer_id'  # Changed from 'lecturer' to 'lecturer_id'
}

df_cleaned = df_cleaned.rename(columns=column_mapping)


In [12]:
# 4. Encode categorical variables
label_encoders = {}
categorical_columns = [
    'program',
    'level',
    'time_pref',
    'subject',
    'lecturer_id'
]

for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    df_cleaned[column] = label_encoders[column].fit_transform(df_cleaned[column])

In [13]:
# 5. Scale numerical features
scaler = StandardScaler()
numerical_columns = [
    'hourly_pay',
    'student_count',
    'credits',
    'institute_rating'
]

df_cleaned[numerical_columns] = scaler.fit_transform(df_cleaned[numerical_columns])

In [14]:
# 6. Prepare features and target
X = df_cleaned.drop('lecturer_id', axis=1)
y = df_cleaned['lecturer_id']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (56, 8)
Testing set shape: (14, 8)


In [15]:
# 7. Save the cleaned data
df_cleaned.to_csv('cleaned_data.csv', index=False)
print("Cleaned data saved to 'cleaned_data.csv'")

Cleaned data saved to 'cleaned_data.csv'


## Data Cleaning Summary

The following steps were performed:
1. Removed duplicate entries
2. Handled missing values
3. Selected relevant features for the model
4. Encoded categorical variables
5. Scaled numerical features
6. Prepared features and target variables
7. Saved the cleaned dataset

The data is now ready for logistic regression modeling.

In [16]:
# At the end of training.ipynb
from save_encoders import save_encoders
save_encoders(label_encoders, scaler)